# Домашнее задание №11.
# Модель Transformer-1

## <a id='Задание_11'>Задание:</a>

Разобраться с моделью перевода (с механизмом внимания) как она устроена, запустить для перевода с русского на английский (при желании можно взять другие пары языков)

## <a id='Подготовка_11'>Подготовка</a>

In [1]:
# Импорт библиотек
import re
import io
import pandas as pd
import transformers
import sacremoses
import corus
import annoy
from transformers import FSMTForConditionalGeneration, FSMTTokenizer

## <a id='Загрузка_данных_11'>Загрузка данных</a>

Используем датасет http://www.manythings.org/anki/

In [2]:
# Скачиваем датасет
!wget http://www.manythings.org/anki/rus-eng.zip

--2023-03-05 12:42:01--  http://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15374406 (15M) [application/zip]
Saving to: ‘rus-eng.zip’

     0K .......... .......... .......... .......... ..........  0%  129K 1m56s
    50K .......... .......... .......... .......... ..........  0%  257K 87s
   100K .......... .......... .......... .......... ..........  0%  139M 58s
   150K .......... .......... .......... .......... ..........  1%  256K 58s
   200K .......... .......... .......... .......... ..........  1%  115M 46s
   250K .......... .......... .......... .......... ..........  1%  596M 38s
   300K .......... .......... .......... .......... ..........  2%  234M 33s
   350K .......... .......... .......... .......... ..........  2%  257K 36s
   400K .......... .......... .......... ....

In [3]:
# Загружаем файл
path_to_file = "/home/jupyter/work/resources/11-Модель_Transformer-1/rus-eng/rus.txt"

In [4]:
# Создаем папку и извлекаем туда файлы из архива
!mkdir rus-eng
!unzip rus-eng.zip -d rus-eng/

Archive:  rus-eng.zip
  inflating: rus-eng/rus.txt         
  inflating: rus-eng/_about.txt      


In [5]:
!ls /home/jupyter/work/resources/11-Модель_Transformer-1/rus-eng/ -lah

total 74M
drwxr-xr-x 1 jupyter jupyter   34 Mar  5 12:42 .
drwxr-xr-x 1 jupyter jupyter  182 Mar  5 12:42 ..
-rw-r--r-- 1 jupyter jupyter 1.5K Feb 20 22:16 _about.txt
-rw-r--r-- 1 jupyter jupyter  74M Feb 20 22:16 rus.txt


In [6]:
# Просмотр файла
f = open(path_to_file)
for line in f:
    print(line)
    break

Go.	Марш!	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1159202 (shanghainese)



## <a id='Подготовка_данных_11'>Подготовка данных</a>

In [7]:
# Функция препроцессинга слов
def preprocess_sentence(w):
  # Переводим предложение к нижнему регистру и удалем начальные и конечные пробелы
    w = w.lower().strip()

  # Отделяем пробелом слово и следующую за ним пунктуацию
  # пример: "he is a boy." => "he is a boy ."
    w = re.sub(r"([?.!,])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

  # Всё, кроме букв и знаков пунктуации, заменяем пробелом
    w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)
  
  # Удаляем лишние пробелы в начале и конце
    w = w.strip()

    return w

In [8]:
# Функция создания датасета
def create_dataset(path, num_examples):
  # Считываем строки файла
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
  # Каждую строку разделяем на пробелы, берем первые 2 слова, препроцессим их и возвращаем пару
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

    return zip(*word_pairs)

In [9]:
# Пример работы
en, ru = create_dataset(path_to_file, None)
print(en[1])
print(ru[1])

go .
иди .


## <a id='Предобученный_трансформер_11'>Предобученная трансформерная архитектура для задачи перевода</a>

In [10]:
mname = "facebook/wmt19-ru-en"
# Создаем экземпляр токенайзера
tokenizer = FSMTTokenizer.from_pretrained(mname)
# Создаем экземпляр модели
model = FSMTForConditionalGeneration.from_pretrained(mname)

In [11]:
# Пример работы с датасетом
for i in range(10):
    input = ru[i]
  # Индексы токенов
    input_ids = tokenizer.encode(input, return_tensors="pt")
  # Выход модели
    output = model.generate(input_ids)
  # Генерация текста
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f'ru: {input}, en: {decoded}, real: {en[i]}')

ru: марш !, en: march!, real: go .
ru: иди ., en: Go., real: go .
ru: идите ., en: Go., real: go .
ru: здравствуйте ., en: Hello., real: hi .
ru: привет !, en: Hello!, real: hi .
ru: хай ., en: Hai., real: hi .
ru: здрасте ., en: , real: hi .
ru: здоро во !, en: Good morning and welcome!, real: hi .
ru: приветик !, en: Hello!, real: hi .
ru: беги !, en: Run!, real: run !


In [ ]:
# Входные данные из домашнего задания 10
inputs = ['хорошая погода', 'я иду в школу', 'Я иду на урок в школу', 
          'Я сам решаю трудную задачу в школе', 'Я не люблю, когда идет снег.', 
          'Сегодня я пойду гулять, а потом пойду в магазин', 'Мама приготовила вкусный обед для нашей большой семьи',
          'Мне здесь нравится, но я очень скучаю по дому. Там остались мама и папа.']
for input in inputs:
  # Индексы токенов
    input_ids = tokenizer.encode(input, return_tensors="pt")
  # Выход модели
    output = model.generate(input_ids)
  # Генерация текста
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f'ru: {input}, en: {decoded}')

**Вывод:** модель справилась с переводом гораздо лучше, чем модели из домашнего задания 10.